## Imports 


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import shapiro
from scipy.stats import normaltest
from sklearn.preprocessing import StandardScaler, RobustScaler
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm
import datetime
sns.set(style="whitegrid")

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

In [3]:
df = pd.read_csv('pek-sha.csv')

In [4]:
df

,ID,flightNumber,craftTypeCode,depAirport,traAirport,arrAirport,departureDate,arrivalDate,cabinClass,priceClass,price,rate,createDate,dateDifference
0,14393,HO1252,320,PEK,NaN,SHA,2019-01-04 06:35:00,2019-01-04 08:55:00,C,C,1860,1.00,2019-01-03 14:26:15,1
1,14409,MU5138,33L,PEK,NaN,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,I,1640,0.31,2019-01-03 14:26:15,1
2,14415,MU5138,33L,PEK,NaN,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,J,5360,1.00,2019-01-03 14:26:15,1
3,14429,HU7605,350,PEK,NaN,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1635,0.29,2019-01-03 14:26:15,1
4,14431,HU7605,350,PEK,NaN,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1640,0.29,2019-01-03 14:26:15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300331,80162962,MU5390,325,PEK,LYA,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,830,0.56,2019-07-23 05:58:47,7
300332,80162963,MU5390,325,PEK,LYA,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,880,0.60,2019-07-23 05:58:47,7
300333,80162964,MU5390,325,PEK,LYA,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,920,0.62,2019-07-23 05:58:47,7
300334,80162965,MU5390,325,PEK,LYA,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,810,0.55,2019-07-23 05:58:47,7


## data cleaning

In [5]:
df.head()

,ID,flightNumber,craftTypeCode,depAirport,traAirport,arrAirport,departureDate,arrivalDate,cabinClass,priceClass,price,rate,createDate,dateDifference
0,14393,HO1252,320,PEK,NaN,SHA,2019-01-04 06:35:00,2019-01-04 08:55:00,C,C,1860,1.00,2019-01-03 14:26:15,1
1,14409,MU5138,33L,PEK,NaN,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,I,1640,0.31,2019-01-03 14:26:15,1
2,14415,MU5138,33L,PEK,NaN,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,J,5360,1.00,2019-01-03 14:26:15,1
3,14429,HU7605,350,PEK,NaN,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1635,0.29,2019-01-03 14:26:15,1
4,14431,HU7605,350,PEK,NaN,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1640,0.29,2019-01-03 14:26:15,1


In [6]:
## fill Nan's with 0 in the transit column 

df.traAirport.fillna(0, inplace = True)

In [7]:
df.traAirport.replace('LYA',1, inplace = True)

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
## , kind="reg"
sns.scatterplot(x='priceClass', y='price', data=df)

In [ ]:
df['price_Class'].sort_values()

In [8]:
df['TOD']= [x[11:16] for x in df.departureDate]

In [9]:
df['TOD'] = pd.to_datetime(df['TOD'])



In [10]:
df['TOD'] = [time.time() for time in df['TOD']]

In [11]:
df['TODD']= np.where(((df['TOD']> datetime.time(6,0)) & (df['TOD']<= datetime.time(12,0))), 'M', np.nan)

In [12]:
df['TODD'] = np.where(((df['TOD']> datetime.time(12,0)) & (df['TOD']<= datetime.time(18,0))), 'A', df['TODD'])

In [13]:
df['TODD'] = np.where(((df['TOD']> datetime.time(18,0)) & (df['TOD']<= datetime.time(23,0))), 'N', df['TODD'])

In [14]:
df

,ID,flightNumber,craftTypeCode,depAirport,traAirport,arrAirport,departureDate,arrivalDate,cabinClass,priceClass,price,rate,createDate,dateDifference,TOD,TODD
0,14393,HO1252,320,PEK,0,SHA,2019-01-04 06:35:00,2019-01-04 08:55:00,C,C,1860,1.00,2019-01-03 14:26:15,1,06:35:00,M
1,14409,MU5138,33L,PEK,0,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,I,1640,0.31,2019-01-03 14:26:15,1,07:00:00,M
2,14415,MU5138,33L,PEK,0,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,J,5360,1.00,2019-01-03 14:26:15,1,07:00:00,M
3,14429,HU7605,350,PEK,0,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1635,0.29,2019-01-03 14:26:15,1,07:20:00,M
4,14431,HU7605,350,PEK,0,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1640,0.29,2019-01-03 14:26:15,1,07:20:00,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300331,80162962,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,830,0.56,2019-07-23 05:58:47,7,13:10:00,A
300332,80162963,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,880,0.60,2019-07-23 05:58:47,7,13:10:00,A
300333,80162964,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,920,0.62,2019-07-23 05:58:47,7,13:10:00,A
300334,80162965,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,810,0.55,2019-07-23 05:58:47,7,13:10:00,A


In [ ]:
sns.barplot(x = df['TODD'], y = df['price'], data= df)

In [15]:
## Get the 
df['DD']= [x[:10] for x in df.departureDate]

In [16]:
df['DD']

0         2019-01-04
1         2019-01-04
2         2019-01-04
3         2019-01-04
4         2019-01-04
             ...    
300331    2019-07-30
300332    2019-07-30
300333    2019-07-30
300334    2019-07-30
300335    2019-07-30
Name: DD, Length: 300336, dtype: object

In [ ]:
sns.barplot(x = df['Month'], y = df['price'], data= df)

In [20]:
import calendar 

In [21]:
def findDay(date):
    born = datetime.datetime.strptime(date, '%Y-%m-%d').weekday()
    return (calendar.day_name[born])

In [22]:
df['DOW'] = [findDay(x) for x in df['DD']]

In [23]:
df

,ID,flightNumber,craftTypeCode,depAirport,traAirport,arrAirport,departureDate,arrivalDate,cabinClass,priceClass,price,rate,createDate,dateDifference,TOD,TODD,DD,Month,DOW
0,14393,HO1252,320,PEK,0,SHA,2019-01-04 06:35:00,2019-01-04 08:55:00,C,C,1860,1.00,2019-01-03 14:26:15,1,06:35:00,M,2019-01-04,20,Friday
1,14409,MU5138,33L,PEK,0,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,I,1640,0.31,2019-01-03 14:26:15,1,07:00:00,M,2019-01-04,20,Friday
2,14415,MU5138,33L,PEK,0,SHA,2019-01-04 07:00:00,2019-01-04 09:15:00,C,J,5360,1.00,2019-01-03 14:26:15,1,07:00:00,M,2019-01-04,20,Friday
3,14429,HU7605,350,PEK,0,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1635,0.29,2019-01-03 14:26:15,1,07:20:00,M,2019-01-04,20,Friday
4,14431,HU7605,350,PEK,0,SHA,2019-01-04 07:20:00,2019-01-04 09:35:00,C,I,1640,0.29,2019-01-03 14:26:15,1,07:20:00,M,2019-01-04,20,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300331,80162962,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,830,0.56,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday
300332,80162963,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,880,0.60,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday
300333,80162964,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,920,0.62,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday
300334,80162965,MU5390,325,PEK,1,SHA,2019-07-30 13:10:00,2019-07-30 18:10:00,Y,R,810,0.55,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday


In [ ]:
sns.barplot(x = df['DOW'], y = df['price'], data= df)

In [ ]:
df['DOW'].value_counts()

In [24]:
df['US_Price'] = [x * 0.14 for x in df['price']]

In [48]:
df.drop(['craftTypeCode', 'departureDate', 'arrivalDate'], axis =1, inplace = True)

In [49]:
df['DOPD'] = [x[:10] for x in df['createDate']]

In [51]:
df['PD'] = [x[:10] for x in df['createDate']]

In [53]:
df['DOPD'] = [findDay(x) for x in df['DOPD']]

In [55]:
df['MOPD'] = [x[5:7] for x in df['PD']]

In [58]:
df['TOPD'] = [x[11:16] for x in df['createDate']]

In [60]:
df['TOPD'] = pd.to_datetime(df['TOPD'])

In [61]:
df['TOPD'] = [time.time() for time in df['TOPD']]

In [62]:
df['TOPD_L']= np.where(((df['TOPD']> datetime.time(6,0)) & (df['TOPD']<= datetime.time(12,0))), 'M', np.nan)
df['TOPD_L'] = np.where(((df['TOPD']> datetime.time(12,0)) & (df['TOPD']<= datetime.time(18,0))), 'A', df['TOPD_L'])
df['TOPD_L'] = np.where(((df['TOPD']> datetime.time(18,0)) & (df['TOPD']<= datetime.time(23,0))), 'N', df['TOPD_L'])

In [63]:
df

,ID,flightNumber,depAirport,traAirport,arrAirport,cabinClass,priceClass,price,rate,createDate,dateDifference,TOD,TODD,DD,Month,DOW,US_Price,DOPD,PD,MOPD,TOPD,TOPD_L
0,14393,HO1252,PEK,0,SHA,C,C,1860,1.00,2019-01-03 14:26:15,1,06:35:00,M,2019-01-04,20,Friday,260.4,Thursday,2019-01-03,01,14:26:00,A
1,14409,MU5138,PEK,0,SHA,C,I,1640,0.31,2019-01-03 14:26:15,1,07:00:00,M,2019-01-04,20,Friday,229.6,Thursday,2019-01-03,01,14:26:00,A
2,14415,MU5138,PEK,0,SHA,C,J,5360,1.00,2019-01-03 14:26:15,1,07:00:00,M,2019-01-04,20,Friday,750.4,Thursday,2019-01-03,01,14:26:00,A
3,14429,HU7605,PEK,0,SHA,C,I,1635,0.29,2019-01-03 14:26:15,1,07:20:00,M,2019-01-04,20,Friday,228.9,Thursday,2019-01-03,01,14:26:00,A
4,14431,HU7605,PEK,0,SHA,C,I,1640,0.29,2019-01-03 14:26:15,1,07:20:00,M,2019-01-04,20,Friday,229.6,Thursday,2019-01-03,01,14:26:00,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300331,80162962,MU5390,PEK,1,SHA,Y,R,830,0.56,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday,116.2,Tuesday,2019-07-23,07,05:58:00,nan
300332,80162963,MU5390,PEK,1,SHA,Y,R,880,0.60,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday,123.2,Tuesday,2019-07-23,07,05:58:00,nan
300333,80162964,MU5390,PEK,1,SHA,Y,R,920,0.62,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday,128.8,Tuesday,2019-07-23,07,05:58:00,nan
300334,80162965,MU5390,PEK,1,SHA,Y,R,810,0.55,2019-07-23 05:58:47,7,13:10:00,A,2019-07-30,20,Tuesday,113.4,Tuesday,2019-07-23,07,05:58:00,nan


In [26]:
df1 = pd.read_csv('sha-pek.csv')

## Second Data set

In [27]:
df1

,ID,flightNumber,craftTypeCode,depAirport,traAirport,arrAirport,departureDate,arrivalDate,cabinClass,priceClass,price,rate,createDate,dateDifference
0,78246995,CA1550,77W,SHA,NaN,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,Z,2980,0.52,2019-07-17 02:32:27,-1
1,78246997,CA1550,77W,SHA,NaN,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,J,5810,1.00,2019-07-17 02:32:27,-1
2,78246998,CA1550,77W,SHA,NaN,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,D,3730,0.65,2019-07-17 02:32:27,-1
3,78246999,CA1550,77W,SHA,NaN,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,C,4170,0.72,2019-07-17 02:32:27,-1
4,78247002,MU5125,33L,SHA,NaN,PEK,2019-07-16 20:05:00,2019-07-16 22:10:00,C,I,2000,0.36,2019-07-17 02:32:27,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268576,77617774,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,830,0.56,2019-07-14 23:28:01,7
268577,77617775,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,880,0.60,2019-07-14 23:28:01,7
268578,77617776,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,920,0.62,2019-07-14 23:28:01,7
268579,77617777,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,790,0.54,2019-07-14 23:28:01,7


In [28]:
df1.traAirport.fillna(0, inplace = True)

In [30]:
df.traAirport.replace('LYA',1, inplace = True)

In [31]:
df1['TOD']= [x[11:16] for x in df1.departureDate]

In [34]:
df1['TOD'] = pd.to_datetime(df1['TOD'])


In [35]:
df1['TOD'] = [time.time() for time in df1['TOD']]

In [76]:
df1['DD'] = [x[:10]for x in df1['departureDate']]

In [77]:
df1['DOW'] = [findDay(x) for x in df1['DD']]

In [36]:
df1['TODD']= np.where(((df1['TOD']> datetime.time(6,0)) & (df1['TOD']<= datetime.time(12,0))), 'M', np.nan)

In [38]:
df1['TODD'] = np.where(((df1['TOD']> datetime.time(12,0)) & (df1['TOD']<= datetime.time(18,0))), 'A', df1['TODD'])

In [39]:
df1['TODD'] = np.where(((df1['TOD']> datetime.time(18,0)) & (df1['TOD']<= datetime.time(23,0))), 'N', df1['TODD'])

In [66]:
df1['US_Price'] = [x * 0.14 for x in df1['price']]

In [67]:
df1['DOPD'] = [x[:10] for x in df1['createDate']]

In [68]:
df1['PD'] = [x[:10] for x in df1['createDate']]

In [69]:
df1['DOPD'] = [findDay(x) for x in df1['DOPD']]

In [70]:
df1['MOPD'] = [x[5:7] for x in df1['PD']]

In [71]:
df1['TOPD'] = [x[11:16] for x in df1['createDate']]

In [72]:
df1['TOPD'] = pd.to_datetime(df1['TOPD'])

In [73]:
df1['TOPD'] = [time.time() for time in df1['TOPD']]

In [74]:
df1['TOPD_L']= np.where(((df1['TOPD']> datetime.time(6,0)) & (df1['TOPD']<= datetime.time(12,0))), 'M', np.nan)
df1['TOPD_L'] = np.where(((df1['TOPD']> datetime.time(12,0)) & (df1['TOPD']<= datetime.time(18,0))), 'A', df1['TOPD_L'])
df1['TOPD_L'] = np.where(((df1['TOPD']> datetime.time(18,0)) & (df1['TOPD']<= datetime.time(23,0))), 'N', df1['TOPD_L'])

In [78]:
df1

,ID,flightNumber,craftTypeCode,depAirport,traAirport,arrAirport,departureDate,arrivalDate,cabinClass,priceClass,price,rate,createDate,dateDifference,TOD,TODD,US_Price,DOPD,PD,MOPD,TOPD,TOPD_L,DD,DOW
0,78246995,CA1550,77W,SHA,0,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,Z,2980,0.52,2019-07-17 02:32:27,-1,19:55:00,N,417.2,Wednesday,2019-07-17,07,02:32:00,nan,2019-07-16,Tuesday
1,78246997,CA1550,77W,SHA,0,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,J,5810,1.00,2019-07-17 02:32:27,-1,19:55:00,N,813.4,Wednesday,2019-07-17,07,02:32:00,nan,2019-07-16,Tuesday
2,78246998,CA1550,77W,SHA,0,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,D,3730,0.65,2019-07-17 02:32:27,-1,19:55:00,N,522.2,Wednesday,2019-07-17,07,02:32:00,nan,2019-07-16,Tuesday
3,78246999,CA1550,77W,SHA,0,PEK,2019-07-16 19:55:00,2019-07-16 22:10:00,C,C,4170,0.72,2019-07-17 02:32:27,-1,19:55:00,N,583.8,Wednesday,2019-07-17,07,02:32:00,nan,2019-07-16,Tuesday
4,78247002,MU5125,33L,SHA,0,PEK,2019-07-16 20:05:00,2019-07-16 22:10:00,C,I,2000,0.36,2019-07-17 02:32:27,-1,20:05:00,N,280.0,Wednesday,2019-07-17,07,02:32:00,nan,2019-07-16,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268576,77617774,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,830,0.56,2019-07-14 23:28:01,7,07:20:00,M,116.2,Sunday,2019-07-14,07,23:28:00,nan,2019-07-21,Sunday
268577,77617775,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,880,0.60,2019-07-14 23:28:01,7,07:20:00,M,123.2,Sunday,2019-07-14,07,23:28:00,nan,2019-07-21,Sunday
268578,77617776,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,920,0.62,2019-07-14 23:28:01,7,07:20:00,M,128.8,Sunday,2019-07-14,07,23:28:00,nan,2019-07-21,Sunday
268579,77617777,MU5389,325,SHA,LYA,PEK,2019-07-21 07:20:00,2019-07-21 12:00:00,Y,R,790,0.54,2019-07-14 23:28:01,7,07:20:00,M,110.6,Sunday,2019-07-14,07,23:28:00,nan,2019-07-21,Sunday


In [44]:
df1['TOD'] = [time.time() for time in df1['TOD']]

AttributeError: 'datetime.time' object has no attribute 'time'

# 